In [24]:
from colored import fg, bg, attr
import argparse
import time

starttime = time.time()

print(sys.version_info)

sys.version_info(major=3, minor=8, micro=5, releaselevel='final', serial=0)


In [25]:
rules = {
"Iron Ore":{"rtyp":"RawMat","T":0.5,"Ab":"IRO","harvest":"Iron Ore Patch","meth":"Harvest,Miner"},
"Copper Ore":{"rtyp":"RawMat","T":0.5,"Ab":"CPO","harvest":"Copper Ore Patch","meth":"Harvest,Miner"},
"Stone":{"rtyp":"RawMat","T":0.5,"Ab":"STO","harvest":"Stone Patch","meth":"Harvest,Miner"},
"Wood":{"rtyp":"RawMat","T":0.5,"Ab":"WOD","harvest":"Woods","meth":"Harvest"},
"Coal":{"rtyp":"RawMat","T":0.5,"Ab":"COL","harvest":"Coal Patch","meth":"Harvest,Miner"},
"Oil":{"rtyp":"RawFluid","T":0.5,"Ab":"OIL","harvest":"Oil Patch","meth":"Oil Pump"},

"Iron Plates":{"rtyp":"Item","T":3.2,"Ab":"IPL","meth":"Furnace","Req":{"IRO":1}},
"Copper Plates":{"rtyp":"Item","T":3.2,"Ab":"CPL","meth":"Furnace","Req":{"CPO":1}},

"Gears":{"rtyp":"Item","T":0.5,"Ab":"GER","meth":"Craft,ARBG","Req":{"IPL":2}},
"Pipes":{"rtyp":"Item","T":0.5,"Ab":"PIP","meth":"Craft,ARBG","Req":{"IPL":1}},

"Coil":{"rtyp":"Item","T":0.5,"Ab":"COI","meth":"Craft,ARBG","Req":{"CPL":0.5}},
"Yellow Belt":{"rtyp":"Item","T":0.5,"Ab":"YBT","meth":"Craft,ARBG","Req":{"IPL":0.5,"GER":0.5}},
"Green Circuits":{"rtyp":"Item","T":0.5,"Ab":"GCI","meth":"Craft,ARBG","Req":{"IPL":1,"COI":1}},

"Offshore Pump":{"rtyp":"Item","T":0.5,"Ab":"OFP","meth":"Craft,ARBG","Req":{"GER":1,"PIP":1,"GCI":2}},
"Stone Furnace":{"rtyp":"Item","T":0.5,"Ab":"STF","meth":"Craft,ARBG","Req":{"STO":5}},
"Boiler":{"rtyp":"Item","T":0.5,"Ab":"BOL","meth":"Craft,ARBG","Req":{"PIP":4,"STF":1}},
"Steam Engine":{"rtyp":"Item","T":0.5,"Ab":"SEG","meth":"Craft,ARBG","Req":{"IPL":10,"GER":8,"PIP":5}},
"Small PowerPole":{"rtyp":"Item","T":0.5,"Ab":"SPP","meth":"Craft,ARBG","Req":{"WOD":1,"CPL":1}},

"Lab":{"rtyp":"Item","T":2,"Ab":"LAB","meth":"Craft,ARBG","Req":{"YBT":4,"GER":10,"GCI":10}},
"Red Science":{"rtyp":"Item","T":5,"Ab":"RSI","meth":"Craft,ARBG","Req":{"CPL":1,"GER":1}},
"Automation Research":{"rtyp":"Item","T":5,"Ab":"ATR","meth":"Lab","Req":{"RSI":10,"OFP":1,"BOL":1,"SEG":1,"SPP":1,"LAB":1}},

"Assembler 1":{"rtyp":"Item","T":5,"Ab":"AS1","meth":"Craft,ARBG","Req":{"IPL":9,"GER":5,"GCI":3,"ATR":1}},
}

In [27]:
class RulingClass:

    abdict = {}
    reqlist = []

    def __init__(self):

        # make abreviation lookup dict
        i = 0
        self.abdict = {}
        for itemname,rule in rules.items():
            ab = rule["Ab"]
            self.abdict[ab] = rule
            rule["name"] = itemname
            print(f"{i} {itemname} {ab} len:{len(self.abdict)}")
            i += 1

        # check requirements correctness
        nbad = 0
        nok = 0
        for itemname,rule in rules.items():
            rtyp = rule["rtyp"]
            if rtyp=="Item":
                req = rule["Req"]
                for reqab,reqquant in req.items():
                    if not reqab in self.abdict:
                        print(f"Error - bad abreviation {reqab} in rule {itemname}")
                        nbad += 1
                    else:
                        nok += 1
        print(f"Checked rule abreviations in reqs nok:{nok} nbad:{nbad}")

    def add_rule_to_list(self,rule):
        self.reqlist.append(rule)
        if not "Req" in rule:
            # must be a raw material
            return
        req = rule["Req"]
        for reqab,reqquant in req.items():
            if not reqab in self.abdict:
                rulename = rule["name"]
                raise Exception(f"Error - bad abreviation {reqab} in rule {rulename}")
            reqrule = self.abdict[reqab]
            if reqquant==0.5:
                reqquant = 1
            for i in range(0,reqquant):
                self.add_rule_to_list(reqrule)        

    def make_req_list(self, req:str):

        if not req in self.abdict:
            raise Exception("req not in abdict")

        start_rule = self.abdict[req]

        self.reqlist = []
        self.add_rule_to_list(start_rule)        

In [29]:
rc = RulingClass()

makename = "AS1"
rc.make_req_list("AS1")

ncraft = 0
nfurnace = 0
nraw = 0
nironplate = 0
ncopperplate = 0
nironore = 0
ncopperore = 0
nstone = 0
idx = 0
for r in rc.reqlist:

    meth = r["meth"]
    if "Craft" in meth:
        ncraft += 1
    elif "Furnace" in meth:
        nfurnace += 1
    else:
        nraw += 1

    if r["rtyp"]=="Item":
        if r["Ab"]=="IPL":
            nironplate += 1
        if r["Ab"]=="CPL":
            ncopperplate += 1

    if r["rtyp"]=="RawMat":
        if r["Ab"]=="IRO":
            nironore += 1
        if r["Ab"]=="CPO":
            ncopperore += 1
        if r["Ab"]=="STO":
            nstone += 1            


    color = bg('navy_blue') + fg('white')
    reset = attr('reset')
    if "Craft" in r["meth"]:
        color = bg('indian_red_1a') + fg('white')
    
    idx += 1
    print(color,idx," ", r, reset)

0 Iron Ore IRO len:1
1 Copper Ore CPO len:2
2 Stone STO len:3
3 Wood WOD len:4
4 Coal COL len:5
5 Oil OIL len:6
6 Iron Plates IPL len:7
7 Copper Plates CPL len:8
8 Gears GER len:9
9 Pipes PIP len:10
10 Coil COI len:11
11 Yellow Belt YBT len:12
12 Green Circuits GCI len:13
13 Offshore Pump OFP len:14
14 Stone Furnace STF len:15
15 Boiler BOL len:16
16 Steam Engine SEG len:17
17 Small PowerPole SPP len:18
18 Lab LAB len:19
19 Red Science RSI len:20
20 Automation Research ATR len:21
21 Assembler 1 AS1 len:22
Checked rule abreviations in reqs nok:35 nbad:0


AttributeError: 'RulingClass' object has no attribute 'add_rule_to_list'

In [ ]:
print(f"to make {makename} you need craft:{ncraft} furnace:{nfurnace} raw:{nraw}")
print(f"to make {makename} you need iron-plate:{nironplate} copper-plate:{ncopperplate}")
print(f"to make {makename} you need iron-ore:{nironore} copper-ore:{ncopperore}  stone:{nstone}")


print(f"to make {makename} you need req_list_len:{len(rc.reqlist)}")


: 

: 

: 

: 